# react_architecture walkthrough（细讲版）

## 目标与先修
- 目标：理解 MPA / SPA(CSR) / SSR / SSG 的数据流差异，并映射到 trace 结论。
- 先修：了解前端渲染时机、首屏性能、SEO 基础概念。


## 流程总览
1. 构建架构矩阵。
2. 逐行拆解四种架构。
3. 运行 `run_demo` 获取统一结论。


### 环境检查（离线模块）
- 本步做什么：确认本模块不依赖外部 API key，直接验证本地依赖可导入。
- 为什么这样做：保证 notebook 能在离线环境稳定 Run All。
- 输入：本地 Python 运行环境与项目依赖。
- 输出：导入成功提示。
- 观察点：如果失败，优先检查 `requirements.txt` 和当前虚拟环境。


In [ ]:
from component import build_architecture_matrix, run_demo

print("环境检查通过：离线依赖已成功导入")


### 工具单元（统一 helper）
- 本步做什么：定义 `show_json` 与 `show_kv`，用于稳定展示中间对象。
- 为什么这样做：教学时必须让过程可观测，避免“函数跑完但看不到内部结构”。
- 输入：任意 Python 对象。
- 输出：可读的 JSON 字符串或键值对列表。
- 观察点：后续每步都复用这两个 helper，输出格式保持一致。


In [ ]:
import json


def show_json(obj):
    print(json.dumps(obj, ensure_ascii=False, indent=2, default=str))


def show_kv(title, mapping):
    print(title)
    for key, value in mapping.items():
        print(f"- {key}: {value}")


## 步骤拆解（逐步）

### Step 1: 生成架构矩阵全表
- 本步做什么：调用 `build_architecture_matrix()`。
- 为什么这样做：先看全局，再逐行对比细节。
- 输入：无。
- 输出：包含四种架构的列表。
- 观察点：重点字段通常包括渲染位置、首屏速度、SEO 与部署特征。


In [ ]:
matrix = build_architecture_matrix()
print("架构数量:", len(matrix))
show_json(matrix)


### Step 2: 拆解 MPA 行
- 本步做什么：提取 `architecture == "MPA"` 的条目。
- 为什么这样做：MPA 是理解其他模式的对照基线。
- 输入：`matrix`。
- 输出：`mpa_row`。
- 观察点：关注页面切换方式与数据获取特点。


In [ ]:
mpa_row = next(item for item in matrix if item["architecture"] == "MPA")
show_json(mpa_row)


### Step 3: 拆解 SPA(CSR) 行
- 本步做什么：提取 `architecture == "SPA_CSR"` 条目。
- 为什么这样做：CSR 在交互体验和首屏/SEO 之间有典型权衡。
- 输入：`matrix`。
- 输出：`csr_row`。
- 观察点：看浏览器端渲染和接口请求时机。


In [ ]:
csr_row = next(item for item in matrix if item["architecture"] == "SPA_CSR")
show_json(csr_row)


### Step 4: 拆解 SSR 行
- 本步做什么：提取 `architecture == "SSR"` 条目。
- 为什么这样做：SSR 常用于提升首屏与 SEO，需要理解服务端成本。
- 输入：`matrix`。
- 输出：`ssr_row`。
- 观察点：关注首屏速度与服务端渲染开销之间的平衡。


In [ ]:
ssr_row = next(item for item in matrix if item["architecture"] == "SSR")
show_json(ssr_row)


### Step 5: 拆解 SSG 行
- 本步做什么：提取 `architecture == "SSG"` 条目。
- 为什么这样做：SSG 是“构建时生成”的典型，适合高缓存命中场景。
- 输入：`matrix`。
- 输出：`ssg_row`。
- 观察点：关注构建成本、更新策略与运行时性能。


In [ ]:
ssg_row = next(item for item in matrix if item["architecture"] == "SSG")
show_json(ssg_row)


## 端到端结果

### Step 6: 运行 run_demo 并映射 trace 结论
- 本步做什么：调用 `run_demo()`，提取 trace 与 final answer。
- 为什么这样做：把“静态矩阵理解”映射到“动态讲解结论”。
- 输入：无。
- 输出：`demo_result`。
- 观察点：trace 应覆盖四种架构对比的关键点。


In [ ]:
demo_result = run_demo()
show_json(demo_result)

print()
print("TRACE EVENTS:")
print([event["event"] for event in demo_result["trace"]])


## 常见错误
- 只记结论不看矩阵字段：容易混淆 CSR/SSR/SSG 的适用场景。
- 忽略部署和更新成本：架构选择会偏离真实工程约束。

## 总结
- 架构选择没有绝对最优，关键是按业务目标（首屏、SEO、交互、成本）权衡。
- 先看矩阵，再看 trace，可快速建立系统化判断框架。
